In [27]:
pip install pyarrow


Note: you may need to restart the kernel to use updated packages.


In [28]:
pip install mmh3


Note: you may need to restart the kernel to use updated packages.


In [29]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [30]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MinMaxScaler
from scipy import sparse
import mmh3

In [31]:
if False:
    df = pd.read_parquet("ub_gaming.parquet.gz")


In [32]:

import pyarrow

bucket='smadex.production.ub'
key = 'Data/ub_gaming.parquet.gzip'

df = pd.read_parquet(f's3://{bucket}/{key}')

# Dataset

In [33]:
pd.set_option('display.max_columns', None)
# df = pd.read_parquet("Data/ub_gaming.parquet.gz")

df = df.fillna(np.nan)
df = df.replace('None',np.nan)
df = df.replace('Unknown',np.nan)
df = df.replace('unknown',np.nan)
df = df.replace('none',np.nan)
df = df.replace('UNK-UNK',np.nan)
df = df.replace('Unk-Unk',np.nan)
df = df.replace('unk-unk',np.nan)
no_rows,no_columns = df.shape
no_rows,no_columns
df[['release_year','release_month']] = df.release_date.str.split("_", expand=True)
df['req_date']=pd.to_datetime(df['req_date'], format='%Y-%m-%dT%H:%M:%S.%fZ')

# don't sort values until we have completed transformation
# sort values in df_transformed
# df =df.sort_values(by=['req_date'],ascending=True)
df

,inv_id,inv_address,inv_bundle,advertiser_bundle,country,req_date,dev_os,dev_osv,dev_make,connection_type_id,req_carrier,ad_size,has_sha1,is_rewarded,is_video,line_group_id,req_city,dev_type,dev_model,exchange_id,is_waterfall,region,release_date,release_msrp,inv_category,inv_subcategory,advertiser_category,advertiser_subcategory,converted,release_year,release_month
0,2352984656,samsung.com,com.samsung.android.game.gamehome,in.playsimple.tripcross,US,2022-12-01 00:24:49.467,android,13.0.0,Samsung,8,310-TM1,native,1,0,0,39767,Chicago,smartphone,SM-S908U,184492,NaN,IL,2022_february,1600.0,NaN,NaN,Games,Word,True,2022,february
1,2352984656,samsung.com,com.samsung.android.game.gamehome,in.playsimple.tripcross,US,2022-12-20 23:41:23.633,android,12.0.0,Samsung,7,310-AT1,native,1,0,0,39767,Houston,smartphone,SM-A125U1,184492,NaN,TX,2020_december,215.0,NaN,NaN,Games,Word,True,2020,december
2,2352984656,samsung.com,com.samsung.android.game.gamehome,in.playsimple.tripcross,US,2022-12-06 02:59:57.597,android,12.0.0,Samsung,3,310-CO1,native,1,0,0,39767,Riverdale,smartphone,SM-A136U,184492,NaN,GA,2021_december,250.0,NaN,NaN,Games,Word,True,2021,december
3,2727210915,NaN,com.samsung.android.game.gamehome,in.playsimple.tripcross,US,2022-12-03 17:03:12.035,android,12.0.0,Samsung,4,310-VE1,native,1,0,0,39767,Palatine,smartphone,SM-A426U,220675,NaN,IL,2020_november,432.0,NaN,NaN,Games,Word,True,2020,november
4,2727210915,NaN,com.samsung.android.game.gamehome,in.playsimple.tripcross,US,2022-12-09 18:57:02.163,android,12.0.0,Samsung,4,310-TM1,native,1,0,0,39767,Chicago,smartphone,SM-N981U1,220675,NaN,IL,2020_august,1400.0,NaN,NaN,Games,Word,True,2020,august
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
790248,4942769735,NaN,match3d.triple.bubble.tile.puzzle.block,com.mytona.cookingdiary.android,US,2022-12-30 21:50:21.444,android,10.0.0,Samsung,7,310-AT1,xlarge,1,0,0,41332,NaN,smartphone,SM-G960U,149058,0.0,GA,2018_march,800.0,Games,Puzzle,Games,Simulation,False,2018,march
790249,1575953459,NaN,word.cross.game.wordcrossy.free,com.mytona.cookingdiary.android,US,2022-12-21 03:06:55.124,android,7.1.1,Samsung,3,NaN,leaderboard,1,0,0,41332,Minot,tablet,SM-T560NU,149058,0.0,ND,2015_july,250.0,Games,Word,Games,Simulation,False,2015,july
790250,5084513639,NaN,com.novaglobal.cast.webvideo,com.mistplay.mistplay,US,2022-12-05 12:50:53.414,android,12.0.0,Samsung,3,310-SP1,xlarge,1,0,0,40356,Shelby,smartphone,SM-A136U,149058,0.0,NC,2021_december,250.0,Applications,Tools,Applications,Entertainment,False,2021,december
790251,5281742121,NaN,top.mmshow.cetus,com.mistplay.mistplay,US,2022-12-01 02:56:27.230,android,12.0.0,TCL,4,310-AT1,xlarge,1,0,0,40356,Indianapolis,smartphone,4188C,220954,0.0,IN,2021_october,NaN,Applications,Tools,Applications,Entertainment,False,2021,october


# Timezones

In [34]:
# The get_time_of_day function now uses the pytz.timezone function to get the timezone object for each row's time_zone value
# and then uses this timezone object to convert the timestamp to the local time of the corresponding timezone.

import pytz

def get_time_zone(country_code):
    time_zones = {
        'US': 'America/New_York',
        'GB': 'Europe/London',
        'DE': 'Europe/Berlin',
        'FR': 'Europe/Paris',
        'BR': 'America/Sao_Paulo',
        'TR': 'Europe/Istanbul',
        'PL': 'Europe/Warsaw',
        'MM': 'Asia/Yangon',
        'RU': 'Europe/Moscow',
        'AU': 'Australia/Sydney'
    }
    return time_zones.get(country_code, None)

def get_local_time(date, time_zone):
    date_utc = date.tz_localize('UTC')
    local_time = date_utc.tz_convert(time_zone)
    return local_time

def get_time_of_day(local_time):#date, time_zone):
    # date_utc = date.tz_localize('UTC')
    # local_time = date_utc.tz_convert(time_zone)
    hour = local_time.hour
    if hour < 6:
        return 'late night'
    elif hour >= 6 and hour < 12:
        return 'morning'
    elif hour >= 12 and hour < 17:
        return 'afternoon'
    elif hour >= 17 and hour < 22:
        return 'evening'
    else:
        return 'early night'

df['req_date'] = pd.to_datetime(df['req_date'])
df.set_index('req_date', inplace=True)

df['time_zone'] = df['country'].apply(lambda x: get_time_zone(x))
df['local_time'] = df.apply(lambda row: get_local_time(row.name, pytz.timezone(row['time_zone'])), axis=1)


df['time_of_day'] = df['local_time'].apply(lambda x: get_time_of_day(x))#df.apply(lambda row: get_time_of_day(row.name, pytz.timezone(row['time_zone'])), axis=1)

def convert_day_int(day_int):
    if day_int == 0:
        return 'Monday'
    elif day_int ==1: 
        return 'Tuesday'
    elif day_int ==2:
        return 'Wednesday'
    elif day_int ==3:
        return 'Thursday'
    elif day_int ==4:
        return 'Friday'
    elif day_int ==5:
        return 'Saturday'
    else:
        return 'Sunday'
        
## todo we should also use local time for the day and weekday/weekend
df['day_of_week'] = df.local_time.apply(lambda x: convert_day_int(x.weekday()))#index.day_name()
df['weekday_or_weekend'] = df['day_of_week'].apply(lambda day: 'weekday' if day in ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'] else 'weekend')



In [35]:
df

,inv_id,inv_address,inv_bundle,advertiser_bundle,country,dev_os,dev_osv,dev_make,connection_type_id,req_carrier,ad_size,has_sha1,is_rewarded,is_video,line_group_id,req_city,dev_type,dev_model,exchange_id,is_waterfall,region,release_date,release_msrp,inv_category,inv_subcategory,advertiser_category,advertiser_subcategory,converted,release_year,release_month,time_zone,local_time,time_of_day,day_of_week,weekday_or_weekend
req_date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2022-12-01 00:24:49.467,2352984656,samsung.com,com.samsung.android.game.gamehome,in.playsimple.tripcross,US,android,13.0.0,Samsung,8,310-TM1,native,1,0,0,39767,Chicago,smartphone,SM-S908U,184492,NaN,IL,2022_february,1600.0,NaN,NaN,Games,Word,True,2022,february,America/New_York,2022-11-30 19:24:49.467000-05:00,evening,Wednesday,weekday
2022-12-20 23:41:23.633,2352984656,samsung.com,com.samsung.android.game.gamehome,in.playsimple.tripcross,US,android,12.0.0,Samsung,7,310-AT1,native,1,0,0,39767,Houston,smartphone,SM-A125U1,184492,NaN,TX,2020_december,215.0,NaN,NaN,Games,Word,True,2020,december,America/New_York,2022-12-20 18:41:23.633000-05:00,evening,Tuesday,weekday
2022-12-06 02:59:57.597,2352984656,samsung.com,com.samsung.android.game.gamehome,in.playsimple.tripcross,US,android,12.0.0,Samsung,3,310-CO1,native,1,0,0,39767,Riverdale,smartphone,SM-A136U,184492,NaN,GA,2021_december,250.0,NaN,NaN,Games,Word,True,2021,december,America/New_York,2022-12-05 21:59:57.597000-05:00,evening,Monday,weekday
2022-12-03 17:03:12.035,2727210915,NaN,com.samsung.android.game.gamehome,in.playsimple.tripcross,US,android,12.0.0,Samsung,4,310-VE1,native,1,0,0,39767,Palatine,smartphone,SM-A426U,220675,NaN,IL,2020_november,432.0,NaN,NaN,Games,Word,True,2020,november,America/New_York,2022-12-03 12:03:12.035000-05:00,afternoon,Saturday,weekend
2022-12-09 18:57:02.163,2727210915,NaN,com.samsung.android.game.gamehome,in.playsimple.tripcross,US,android,12.0.0,Samsung,4,310-TM1,native,1,0,0,39767,Chicago,smartphone,SM-N981U1,220675,NaN,IL,2020_august,1400.0,NaN,NaN,Games,Word,True,2020,august,America/New_York,2022-12-09 13:57:02.163000-05:00,afternoon,Friday,weekday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-30 21:50:21.444,4942769735,NaN,match3d.triple.bubble.tile.puzzle.block,com.mytona.cookingdiary.android,US,android,10.0.0,Samsung,7,310-AT1,xlarge,1,0,0,41332,NaN,smartphone,SM-G960U,149058,0.0,GA,2018_march,800.0,Games,Puzzle,Games,Simulation,False,2018,march,America/New_York,2022-12-30 16:50:21.444000-05:00,afternoon,Friday,weekday
2022-12-21 03:06:55.124,1575953459,NaN,word.cross.game.wordcrossy.free,com.mytona.cookingdiary.android,US,android,7.1.1,Samsung,3,NaN,leaderboard,1,0,0,41332,Minot,tablet,SM-T560NU,149058,0.0,ND,2015_july,250.0,Games,Word,Games,Simulation,False,2015,july,America/New_York,2022-12-20 22:06:55.124000-05:00,early night,Tuesday,weekday
2022-12-05 12:50:53.414,5084513639,NaN,com.novaglobal.cast.webvideo,com.mistplay.mistplay,US,android,12.0.0,Samsung,3,310-SP1,xlarge,1,0,0,40356,Shelby,smartphone,SM-A136U,149058,0.0,NC,2021_december,250.0,Applications,Tools,Applications,Entertainment,False,2021,december,America/New_York,2022-12-05 07:50:53.414000-05:00,morning,Monday,weekday


In [36]:
df['req_date'] = df.index
df = df.reset_index(drop=True)
df

,inv_id,inv_address,inv_bundle,advertiser_bundle,country,dev_os,dev_osv,dev_make,connection_type_id,req_carrier,ad_size,has_sha1,is_rewarded,is_video,line_group_id,req_city,dev_type,dev_model,exchange_id,is_waterfall,region,release_date,release_msrp,inv_category,inv_subcategory,advertiser_category,advertiser_subcategory,converted,release_year,release_month,time_zone,local_time,time_of_day,day_of_week,weekday_or_weekend,req_date
0,2352984656,samsung.com,com.samsung.android.game.gamehome,in.playsimple.tripcross,US,android,13.0.0,Samsung,8,310-TM1,native,1,0,0,39767,Chicago,smartphone,SM-S908U,184492,NaN,IL,2022_february,1600.0,NaN,NaN,Games,Word,True,2022,february,America/New_York,2022-11-30 19:24:49.467000-05:00,evening,Wednesday,weekday,2022-12-01 00:24:49.467
1,2352984656,samsung.com,com.samsung.android.game.gamehome,in.playsimple.tripcross,US,android,12.0.0,Samsung,7,310-AT1,native,1,0,0,39767,Houston,smartphone,SM-A125U1,184492,NaN,TX,2020_december,215.0,NaN,NaN,Games,Word,True,2020,december,America/New_York,2022-12-20 18:41:23.633000-05:00,evening,Tuesday,weekday,2022-12-20 23:41:23.633
2,2352984656,samsung.com,com.samsung.android.game.gamehome,in.playsimple.tripcross,US,android,12.0.0,Samsung,3,310-CO1,native,1,0,0,39767,Riverdale,smartphone,SM-A136U,184492,NaN,GA,2021_december,250.0,NaN,NaN,Games,Word,True,2021,december,America/New_York,2022-12-05 21:59:57.597000-05:00,evening,Monday,weekday,2022-12-06 02:59:57.597
3,2727210915,NaN,com.samsung.android.game.gamehome,in.playsimple.tripcross,US,android,12.0.0,Samsung,4,310-VE1,native,1,0,0,39767,Palatine,smartphone,SM-A426U,220675,NaN,IL,2020_november,432.0,NaN,NaN,Games,Word,True,2020,november,America/New_York,2022-12-03 12:03:12.035000-05:00,afternoon,Saturday,weekend,2022-12-03 17:03:12.035
4,2727210915,NaN,com.samsung.android.game.gamehome,in.playsimple.tripcross,US,android,12.0.0,Samsung,4,310-TM1,native,1,0,0,39767,Chicago,smartphone,SM-N981U1,220675,NaN,IL,2020_august,1400.0,NaN,NaN,Games,Word,True,2020,august,America/New_York,2022-12-09 13:57:02.163000-05:00,afternoon,Friday,weekday,2022-12-09 18:57:02.163
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
790248,4942769735,NaN,match3d.triple.bubble.tile.puzzle.block,com.mytona.cookingdiary.android,US,android,10.0.0,Samsung,7,310-AT1,xlarge,1,0,0,41332,NaN,smartphone,SM-G960U,149058,0.0,GA,2018_march,800.0,Games,Puzzle,Games,Simulation,False,2018,march,America/New_York,2022-12-30 16:50:21.444000-05:00,afternoon,Friday,weekday,2022-12-30 21:50:21.444
790249,1575953459,NaN,word.cross.game.wordcrossy.free,com.mytona.cookingdiary.android,US,android,7.1.1,Samsung,3,NaN,leaderboard,1,0,0,41332,Minot,tablet,SM-T560NU,149058,0.0,ND,2015_july,250.0,Games,Word,Games,Simulation,False,2015,july,America/New_York,2022-12-20 22:06:55.124000-05:00,early night,Tuesday,weekday,2022-12-21 03:06:55.124
790250,5084513639,NaN,com.novaglobal.cast.webvideo,com.mistplay.mistplay,US,android,12.0.0,Samsung,3,310-SP1,xlarge,1,0,0,40356,Shelby,smartphone,SM-A136U,149058,0.0,NC,2021_december,250.0,Applications,Tools,Applications,Entertainment,False,2021,december,America/New_York,2022-12-05 07:50:53.414000-05:00,morning,Monday,weekday,2022-12-05 12:50:53.414
790251,5281742121,NaN,top.mmshow.cetus,com.mistplay.mistplay,US,android,12.0.0,TCL,4,310-AT1,xlarge,1,0,0,40356,Indianapolis,smartphone,4188C,220954,0.0,IN,2021_october,NaN,Applications,Tools,Applications,Entertainment,False,2021,october,America/New_York,2022-11-30 21:56:27.230000-05:00,evening,Wednesday,weekday,2022-12-01 02:56:27.230


# Transforming the data

In [37]:
df_transformed = pd.DataFrame()

<h3> Select only relevant values

In [38]:
def relevant_or_none(x,col_name, keep=None):
    if x in keep:
        return x
    else:
        return 'low_frequency_value'+col_name

In [39]:
# leave most relevant for now as we are using hashing trick
# test later to see if there is an improvement on any model

most_relevant_makes = df.dev_make.value_counts()[df.dev_make.value_counts()>50]
df['dev_make'] = df.dev_make.apply(lambda x: relevant_or_none(x,'dev_make',most_relevant_makes))
most_relevant_regions = df.region.value_counts()[df.region.value_counts()>50]
df['region'] = df.region.apply(lambda x: relevant_or_none(x,'region',most_relevant_regions))
most_relevant_models = df.dev_model.value_counts()[df.dev_model.value_counts()>50]
df['dev_model'] = df.dev_model.apply(lambda x: relevant_or_none(x,'dev_model',most_relevant_models))
most_relevant_cities = df.req_city.value_counts()[df.req_city.value_counts()>100]
df['req_city'] = df.req_city.apply(lambda x: relevant_or_none(x,'req_city',most_relevant_cities))

In [40]:
df.nunique()

inv_id                     49251
inv_address                  629
inv_bundle                 19269
advertiser_bundle             21
country                        5
dev_os                         2
dev_osv                      176
dev_make                      68
connection_type_id             8
req_carrier                  131
ad_size                        9
has_sha1                       2
is_rewarded                    2
is_video                       2
line_group_id                 40
req_city                    1001
dev_type                       2
dev_model                    774
exchange_id                   28
is_waterfall                   2
region                       117
release_date                 133
release_msrp                 655
inv_category                  26
inv_subcategory               51
advertiser_category            2
advertiser_subcategory        11
converted                      2
release_year                  15
release_month                 12
time_zone 

In [41]:
df_transformed['is_smartphone'] = df.dev_type.apply(lambda x: 1 if x=='smartphone' else 0)

In [42]:
df_transformed['ios_has_sha'] = df.apply(lambda x: 1 if ((x.dev_os=='ios') & (x.has_sha1==1)) else 0,axis=1)
df_transformed['android_no_sha'] =df.apply(lambda x: 1 if ((x.dev_os=='android') & (x.has_sha1==1)) else 0,axis=1)

<h5> Ordinal Variable

In [43]:
df['release_date'] = pd.to_datetime(df['release_date'], format='%Y_%B')

In [ ]:
def num_months(x):
    if pd.isnull(x):
        return 70  #approximate median
    else:
        return (pd.to_datetime('2023-02-01').to_period('M')- x.to_period('M')).n

df_transformed['release_date']=df.release_date.apply(lambda x: num_months(x))

<h5> Average Pricing

In [ ]:
make_avg_prices = df[['dev_make','release_msrp']].groupby(by=['dev_make'])['release_msrp'].mean()
dev_os_avg_prices = df[['dev_os','release_msrp']].groupby(by=['dev_os'])['release_msrp'].mean()

In [ ]:
def fill_with_average_price(row, make_avg_prices, dev_os_avg_prices):
    if not pd.isnull(row.release_msrp):
        return row.release_msrp
    elif row.dev_make in make_avg_prices:
        return make_avg_prices[row.dev_make]
    elif row.dev_os in dev_os_avg_prices:
        return dev_os_avg_prices[row.dev_os]
    else:
        return np.nan

df_transformed['release_msrp'] = df.apply(
    lambda x: fill_with_average_price(x, make_avg_prices, dev_os_avg_prices),
    axis=1)

# Fill any remaining NaNs with the mean value of the column
df_transformed['release_msrp'] = df_transformed['release_msrp'].fillna(df_transformed['release_msrp'].mean())

In [ ]:
df_transformed['release_msrp'].isna().sum()

0

<h5> Normalize

In [ ]:
scaler = MinMaxScaler()
df_transformed['release_msrp'] = scaler.fit_transform(df_transformed[['release_msrp']])
scaler = MinMaxScaler()
df_transformed['release_date'] = scaler.fit_transform(df_transformed[['release_date']])

<h5> Transform subcategory --> maybe just delete it?

<h3> One Hot Encoding

In [ ]:
categorical_cols = ['dev_os', 'advertiser_bundle', 'ad_size', 'connection_type_id', 'line_group_id', 
                     'dev_make', 'region', 'release_year', 'release_month', 
                    'day_of_week', 'weekday_or_weekend', 'time_zone', 'time_of_day'] 

df['connection_type_id'] = df['connection_type_id'].astype(str)
df['line_group_id'] = df['line_group_id'].astype(str)
df_transformed = pd.concat([df_transformed,pd.get_dummies(df[categorical_cols], drop_first=True)],axis=1)

<h3> Hashing

In [ ]:
hash_columns = ['inv_address', 'inv_bundle', 'country', 'exchange_id', 'inv_category', 'advertiser_category', 'advertiser_subcategory', 'req_city', 'dev_model']
df[hash_columns] = df[hash_columns].fillna("NA")

NUM_ROWS = len(df_transformed)

In [ ]:
OUTPUT_SIZE = 1000
items = dict()
sparse_hashed = sparse.lil_matrix((NUM_ROWS,OUTPUT_SIZE),dtype=np.int8)
for i, row in df.iterrows():
    for col in hash_columns:
        index = mmh3.hash(str(row[col]))
        index = index % OUTPUT_SIZE 
        items[row[col]]  = index
        sparse_hashed[i,index] = 1
sparse_hashed = sparse_hashed.tocsr()
# sparse.save_npz('Data/transformed_data/'+str(OUTPUT_SIZE)+'_sparse_hashed.npz', sparse_hashed)

In [ ]:
total = len(items.values())
unique = len(list(dict.fromkeys(items.values())) )
print(((1-unique/total)*100),'% colisiions')

95.39658426552502 % colisiions


In [ ]:
sparse_hashed_df = pd.DataFrame.sparse.from_spmatrix(sparse_hashed)
sparse_hashed_df.columns = sparse_hashed_df.columns.map(str)
sparse_hashed_df = sparse_hashed_df.sparse.to_dense()
sparse_hashed_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720,721,722,723,724,725,726,727,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,768,769,770,771,772,773,774,775,776,777,778,779,780,781,782,783,784,785,786,787,788,789,790,791,792,793,794,795,796,797,798,799,800,801,802,803,804,805,806,807,808,809,810,811,812,813,814,815,816,817,818,819,820,821,822,823,824,825,826,827,828,829,830,831,832,833,834,835,836,837,838,839,840,841,842,843,844,845,846,847,848,849,850,851,852,853,854,855,856,857,858,859,860,861,862,863,864,865,866,867,868,869,870,871,872,873,874,875,876,877,878,879,880,881,882,883,884,885,886,887,888,889,890,891,892,893,894,895,896,897,898,899,900,901,902,903,904,905,906,907,908,909,910,911,912,913,914,915,916,917,918,919,920,921,922,923,924,925,926,927,928,929,930,931,932,933,934,935,936,937,938,939,940,941,942,943,944,945,946,947,948,949,950,951,952,953,954,955,956,957,958,959,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0

<h4> Dropping

In [ ]:
# DROP req_carrier, is_waterfall, inv_id, req_date, dev_osv, dev_type, has_sha
no_use_col = ['req_carrier', 'is_waterfall', 'inv_id', 'dev_osv', 'dev_type', 'has_sha1', 'inv_subcategory']
df_transformed = df_transformed.drop(no_use_col, axis=1,errors='ignore')

<h4> Sorting

<h3> Final Dataset

In [ ]:
df_transformed['req_date']=df['req_date']
df_transformed['converted'] = df['converted'].apply(lambda x: 1 if x else 0)
# df_transformed.to_csv('Data/transformed_data/non_sparse_transformed_date.csv')

In [ ]:
full = df_transformed.merge(sparse_hashed_df, left_index=True,right_index=True)
cols =full.columns
cols = [i.replace('.','_') for i in cols]
full.columns = cols

df_train = full[full['req_date']<'2022-12-15'].drop(['req_date'],axis=1,inplace=False)
df_not_train = full[full['req_date']>='2022-12-15'].drop(['req_date'],axis=1,inplace=False)
df_not_train_y = df_not_train['converted']

df_train

is_smartphone  ios_has_sha  android_no_sha  release_date  \
0                   1            0               1      0.036364   
2                   1            0               1      0.043636   
3                   1            0               1      0.090909   
4                   1            0               1      0.101818   
9                   1            0               1      0.145455   
...               ...          ...             ...           ...   
790245              1            0               1      0.083636   
790247              1            0               1      0.170909   
790250              1            0               1      0.043636   
790251              1            0               1      0.050909   
790252              1            0               1      0.160000   

        release_msrp  dev_os_ios  advertiser_bundle_1200391796  \
0           0.663721           0                             0   
2           0.095539           0                             0   
3           0.172138           0                             0   
4           0.579545           0                             0   
9           0.495370           0                             0   
...              ...         ...                           ...   
790245      0.130471           0                             0   
790247      0.061869           0                             0   
790250      0.095539           0                             0   
790251      0.059336           0                             0   
790252      0.066077           0                             0   

        advertiser_bundle_1214763610  advertiser_bundle_1269540854  \
0                                  0                             0   
2                                  0                             0   
3                                  0                             0   
4                                  0                             0   
9                                  0                             0   
...                              ...                           ...   
790245                             0                             0   
790247                             0                             0   
790250                             0                             0   
790251                             0                             0   
790252                             0                             0   

        advertiser_bundle_1371565796  advertiser_bundle_1432579280  \
0                                  0                             0   
2                                  0                             0   
3                                  0                             0   
4                                  0                             0   
9                                  0                             0   
...                              ...                           ...   
790245                             0                             0   
790247                             0                             0   
790250                             0                             0   
790251                             0                             0   
790252                             0                             0   

        advertiser_bundle_1433784188  advertiser_bundle_1464675244  \
0                                  0                             0   
2                                  0                             0   
3                                  0                             0   
4                                  0                             0   
9                                  0                             0   
...                              ...                           ...   
790245                             0                             0   
790247                             0                             0   
790250                             0                             0   
790251          

In [ ]:
from sklearn.model_selection import train_test_split
				
df_val, df_test, y_val, y_test = train_test_split(df_not_train, df_not_train_y, test_size=0.5, random_state=42)
df_val['converted'] = y_val
df_test['converted'] = y_test
print('here')
df_train.to_hdf('smadex_train.h5',key='train')#,header=True,index=False)
print('done1')
df_val.to_hdf('smadex_val.h5',key='val')#,header=True,index=False)
print('done2')
df_test.to_hdf('smadex_test.h5',key='test')#,header=True,index=False)
print('done3')

here
done1
done2
done3


In [ ]:
pref = '{name: \"'
suff = '\", active: True, dtype: str, type: categorical},'
print(pref+'xxx'+suff)
cols = df_train.columns.map(str).tolist()
cols.remove('converted')
cols.remove('release_date')
cols.remove('release_msrp')
cols = [pref + sub for sub in cols]
cols = [sub + suff for sub in cols]
with open('columns.txt','w+') as f:
	for i in cols:
		f.write('%s\n'%i)

{name: "xxx", active: True, dtype: str, type: categorical},


In [ ]:
# Instantiate the logistic regression model
from sklearn.linear_model import LogisticRegression
from datetime import datetime

# Extract the target variable from the training dataframe
y_train = df_train['converted']
X_train = df_train.drop('converted', axis=1,inplace=False)



# Instantiate the logistic regression model with class weights
model = LogisticRegression(class_weight={0: 200, 1: 1}, max_iter=9999999)
model_config = str(model.get_params())
# Train the model
start_time = datetime.now()
model.fit(X_train, y_train)
train_time = datetime.now() - start_time


# Extract the target variable from the validation dataframe
y_val = df_val['converted']

# Extract feature columns from the validation dataframe
X_val = df_val.drop('converted', axis=1,inplace=False)



# Make predictions on the validation set

start_time = datetime.now()
y_pred = model.predict_proba(X_val)
pred_time = datetime.now() - start_time

In [ ]:
from dump_results import dump_results

In [ ]:
train_time = str(train_time)
pred_time = str(pred_time)
dump_results(y_val, y_pred[:,1], model_config, train_time, pred_time, 'LogisticRegression')

In [ ]:
import numpy as np
from sklearn.metrics import log_loss
valid_weights = []

y_valid = y_val
for t in y_valid:
    if t == 0:
        valid_weights.append(200)
    else:
        valid_weights.append(1)

def real_prob(k,p):
    if p == 0:
        return 0
    return 1/(1-k+(k/p))
from sklearn.metrics import log_loss
baseline_ll = log_loss(y_valid, [real_prob(200,np.mean(y_valid))]*len(y_valid), sample_weight=valid_weights)
model_ll = log_loss(y_valid, y_pred, sample_weight=valid_weights)
print('NLL: ', 1 - model_ll/baseline_ll)

NLL:  -2.743415266177579


In [ ]:
from sklearn.metrics import mutual_info_score as mi
for col in df_train:
    score =mi(df_train[col],df_train['converted'])
    if score > 10e-4:
        print(col, score)

release_date 0.002061626460129554
release_msrp 0.0050029665302544525


/Users/home/Applications/anaconda3/envs/pytorch/lib/python3.8/site-packages/sklearn/metrics/cluster/_supervised.py:64: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)
/Users/home/Applications/anaconda3/envs/pytorch/lib/python3.8/site-packages/sklearn/metrics/cluster/_supervised.py:64: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)


advertiser_bundle_1214763610 0.001517454058180853
advertiser_bundle_1269540854 0.0021290432334014724
advertiser_bundle_1432579280 0.0015792236524224193
advertiser_bundle_529996768 0.0012137653287414337
advertiser_bundle_561941526 0.0013903562585414302
advertiser_bundle_com_mistplay_mistplay 0.003949097648873503
advertiser_bundle_com_playvalve_watersort 0.004064185347213575
ad_size_interstitial_landscape 0.0013942742393540895
ad_size_native 0.0016834811528678197
ad_size_xlarge 0.008901261201728055
line_group_id_35251 0.0015105158464620554
line_group_id_38367 0.0011847019625073583
line_group_id_39452 0.0013243624723274742
line_group_id_39631 0.0010246705535496541
line_group_id_40340 0.001693790726044777
line_group_id_40346 0.0013461458465493566
line_group_id_40356 0.005081781425988332
line_group_id_40384 0.0021290432334014724
line_group_id_40477 0.0010555675510639717
line_group_id_40598 0.0016288226916179073
line_group_id_40845 0.0013965719356526052
line_group_id_41223 0.0022907212222364

In [ ]:
# best split for timeseries is 70-20-10, maybe decide it on req_date (which is now sorted in ascended order) and slice
# other option could be TimeSeriesSplit from sklearn

In [ ]:
# for unbalanced dataset --> best do ask for more converted data?